### Pokerh&auml;nde
Eine Karte wird als String der L&auml;nge 2 repr&auml;sentiert, z.B. 'A♣'.
Das 1. Zeichen steht f&uuml;r den Rang/rank der Karte und ist eines der Zeichen
in '23456789TJQKA' (zwei,...,neun, ten, jack, queen, king und ace), 
das 2. Zeichen steht f&uuml;r die Farbe/suit der Karte und ist eines der Zeichen in '♥♠♦♣' (hearts, spades, diamonds, clubs).

Der Kartenstapel/deck ist eine Liste mit 52 Karten, eine Hand ist eine Liste mit
5 Karten. Zum mischen des Decks verwenden wir die Funktion `random.shuffle(deck)`,
welche eine Liste mischt.   

Nachstehend definieren wir einige Konstanten, die wir im ganzen Notebook nutzen,
sowie die Funktionen `deal(deck)`, welche die 5 obersten Karten des Decks `deck`
entfernt und als Liste zur&uuml;ck gibt (die gedealte Hand), sowie
`get_ranks(hand)` und `get_suits(hand)`, welche jeweils nur die die Ranks bez. die Suits unserer Handkarten herausgreift (siehe unten).

In [ ]:
import random


RANK = 0
SUIT = 1
SUITS = '♥♠♦♣'
RANKS = '23456789TJQKA'

deck = [value+suit for suit in SUITS for value in RANKS]
card = deck[-1]
card, card[RANK], card[SUIT]

In [ ]:
deck[-13:]

In [ ]:
random.shuffle(deck)
deck[-13:]

In [ ]:
def draw(deck, n=5):
    '''die letzten n Karten vom deck entfernen und
       als Liste zurueckgeben
    '''
    hand = []
    for _ in range(5):
        hand.append(deck.pop())
    return hand

In [ ]:
hand = draw(deck)
hand

In [ ]:
def get_ranks(hand):
    return [card[RANK] for card in hand]


def get_suits(hand):
    return [card[SUIT] for card in hand]

In [ ]:
ranks = get_ranks(hand)
suits = get_suits(hand)
ranks, suits

In [ ]:
# ranks Ordnen bez. 
# der rank einer Karte ist umso besser, je weiter rechts
# er in der Liste RANKS steht, d.h. je groesser RANK.index(rank) ist
sorted(ranks, key=lambda x: RANKS.index(x))

### Handtyp feststellen

Ziel ist es, eine Funktion zu schreiben, die den [Handtyp](https://en.wikipedia.org/wiki/List_of_poker_hands)
(four of a kind, full house, flush, straight, ...) erkennt.
Dann nehmen wir 100_000 Mal die obersten 5 Karten eines gemischten Decks und schauen, wie oft die einzelnen Handtypen vorkommen.  

- flush: falls alle Karten die gleiche Suit haben,
- straight: wir sortieren die Ranks der Handkarten, machen daraus einen String und schauen ob dieser in der Liste STRAIGHTS vorkommt (siehe unten).

Ist eine Hand kein flush oder straight, erstellen wir einen Dictionary, der zu jedem Rank angibt, wieviele Karten mit diesem Rank in der Hand sind.
F&uuml;r eine Hand `['5♠', '4♠', 'A♥', '5♥', '7♥']` sieht dieser Dictionary so aus:
`{'5': 2, 4: '1', 'A': 1, '7': 1}`. Die absteigend sortierten Werte, hier
'2111', nennen wir dann den CTYPE dieser Hand. So eine Hand nennt man auch pair.

In [ ]:
STRAIGHTS = ['2345A', '23456', '34567', '45678', '56789', '6789T',
             '789TJ', '89TJQ', '9TJQK', 'TJQKA',
             ]

CTYPE_NAME = {
    '11111': 'high card',
    '2111': 'pair',
    '221': 'pairs',
    '311': 'trips',
    '32': 'fullhouse',
    '41': 'quads',
}

### Aufgaben

In [ ]:
def is_flush(hand):
    '''True falls die hand ein flush ist'''   
    return len(set(get_suits(hand))) == 1

In [ ]:
hand = ['A♣', '7♣', 'Q♣', '6♣', 'T♣']
is_flush(hand)

In [ ]:
def is_straight(hand):
    '''True falls die hand ein straight ist'''
    ranks = get_ranks(hand)
    ranks_sorted = sorted(ranks, key=lambda x: RANKS.index(x))
    return ''.join(ranks_sorted) in STRAIGHTS

In [ ]:
hands = [['3♣', 'A♥', '2♣', '5♣', '4♣'],
         ['K♣', 'A♣', 'Q♥', 'T♣', 'J♣'],
         ['8♣', 'T♣', 'T♥', '9♥', 'Q♣'],
        ]

[is_straight(h) for h in hands]

In [ ]:
def count_dict(items):
    d = {}
    for item in items:
        d[item] = d.get(item, 0) + 1
    return d


def count_type(hand):
    '''gibt den CTYPE der Hand zur&uuml;ck'''
    ranks = get_ranks(hand)
    d = count_dict(ranks)
    counts = sorted(d.values(), reverse=True)
    return ''.join(str(x) for x in counts)

In [ ]:
hands = [['3♣', 'A♥', '2♥', '5♣', '4♣'],
         ['K♣', 'A♣', 'J♥', 'A♥', 'J♣'],
         ['9♣', 'T♣', 'T♥', '9♥', '9♦'],
        ]

[count_type(h) for h in hands]

In [ ]:
def handname(hand):
    '''gibt den Handnamen der Hand zurueck'''
    if is_straight(hand) and is_flush(hand):
        return 'straightflush'
    elif is_flush(hand):
        return 'flush'
    elif is_straight(hand):
        return 'straight'
    else:
        ct = count_type(hand)
        return CTYPE_NAME[ct]

In [ ]:
hands = [['3♣', 'A♥', '2♥', '5♣', '4♣'],
         ['K♣', 'A♣', 'J♥', 'A♥', 'J♣'],
         ['9♣', 'T♣', 'T♥', '9♥', '9♦'],
        ]

[handname(h) for h in hands]

In [ ]:
# Wieviele Haende eines bestimmten Types
# erhaelt man bei 100_000 Mal ziehen
d = {}
for i in range(100_000):
    deck = new_deck(shuffle=True)
    hand = draw(deck, n=5)
    key = handname(hand)
    d[key] = d.get(key, 0) + 1
d = dict(sorted(d.items(), key=lambda x: x[::-1]))
d